In [27]:
import pandas as pd
import numpy as np
import re
import openpyxl

In [ ]:
# Load Data - M3 Data
part_sales = pd.read_excel("C:/Users/tmanett/OneDrive - Wajax Limited/Desktop/Handheld Rollout/M3 Parts Sales - Query.xlsx")

distribution_data = pd.read_csv("C:/Users/tmanett/OneDrive - Wajax Limited/Desktop/Handheld Rollout/M3 Distribution Orders - Delivered or Received.csv", skiprows=6)

purchase_orders = pd.read_excel("C:/Users/tmanett/OneDrive - Wajax Limited/Desktop/Handheld Rollout/Copy of M3 Purchase Orders - All - Query.xlsx")

part_sales_AX = pd.read_excel("C:/Users/tmanett/OneDrive - Wajax Limited/Desktop/Handheld Rollout/AX Part Sales.xlsx", skiprows=2)

distribution_data_AX = pd.read_csv("C:/Users/tmanett/OneDrive - Wajax Limited/Desktop/Handheld Rollout/distribution_ax.csv", skiprows=3)

In [38]:
purchase_orders_AX = pd.read_excel("C:/Users/tmanett/OneDrive - Wajax Limited/Desktop/Handheld Rollout/AX PO .xlsx", skiprows=2)

### Python Code for Merging the Part Sales with a Left Join

In [17]:
# Rename AX columns to match M3 columns
part_sales_ax_renamed = part_sales_AX.rename(columns={
    'Branch Code - Rev': 'Facility',
    'Branch Name - Rev': 'FacilityName',
    'Sales Qty': 'Qty',
    'Sales$': 'ExtSales',
    'Cost$': 'ExtCostCAD',
    'Gross Margin$': 'GrossMargin',
    'Gross Margin%': 'GrossMarginPct',
    'Net Margin$': 'NetMargin',
    'Net Margin%': 'NetMarginPct',
    'Hits': 'Hits'
})

# Select relevant columns from AX to match the M3 dataset
common_columns = ['Facility', 'FacilityName', 'Qty', 'ExtSales', 'ExtCostCAD', 'GrossMargin', 'GrossMarginPct', 'NetMargin', 'NetMarginPct', 'Hits']
part_sales_ax_renamed = part_sales_ax_renamed[common_columns]

# Merge M3 and AX part sales data using a left join on 'Facility' and 'FacilityName'
combined_part_sales = pd.merge(part_sales, part_sales_ax_renamed, on=['Facility', 'FacilityName'], how='left', suffixes=('_m3', '_ax'))

# Display combined columns to check if they match
print("Combined Columns:")
print(combined_part_sales.columns.tolist())

Combined Columns:
['RunDate', 'RunStartDate', 'RunEndDate', 'Domain', 'Company', 'CompanyName', 'Div', 'DivisionName', 'RegionName', 'Facility', 'FacilityName', 'InvoiceNumber', 'InvoiceDate', 'DeliveryDate', 'OrderType', 'OrderTypeDesc', 'CustomerCode', 'CustomerName', 'CustNDSCustomer', 'DiscountGroup', 'CustGroup', 'CustomerPO', 'Payer', 'PayerName', 'PayerNDSCustomer', 'PayerCstGroup', 'SplitInd', 'DeliveryNumber', 'OrderNumber', 'LineNbr', 'LineSuffix', 'PPRReference', 'AgreemNbr', 'Service', 'ClaimNumber', 'ClaimforOrder', 'ClaimforOrdLine', 'LineType', 'LineReqDelivDate', 'LineConfirmDelivDate', 'Responsible', 'ResponsibleName', 'OrderSalesperson', 'OrderSalespersonName', 'LineSalesperson', 'LineSalespersonName', 'Whse', 'WareHouseName', 'WorkOrder', 'WODescription1', 'WODescription2', 'Product', 'ProductName', 'ProdDesc/Model', 'ProductSerialNbr', 'ProdExtItemNumber', 'FleetNbr', 'ProdItemType', 'ProdItemTypeDesc', 'ProdItemGroup', 'ProdItemGroupDesc', 'ProdItemCat', 'ProdItemC

### Part Sales M3

In [18]:
# Define the list of cities in the GTA region
gta_facilities = ["Toronto", "Mississauga", "Brampton", "Vaughan", "Markham", "Richmond Hill", "Oakville", "Burlington", "Pickering", "Ajax", "Whitby"]

# Function to add the "Middle Gate" branch based on FacilityName
def add_middle_gate_branch(df, facility_column='FacilityName'):
    gta_pattern = '|'.join(gta_facilities)
    middle_gate = df[(df['ItemCat'].str.contains('MAT')) & (df[facility_column].str.contains(gta_pattern))]
    middle_gate.loc[:, facility_column] = 'Middle Gate'  # Use .loc to avoid SettingWithCopyWarning
    df_without_gta = df[~((df['ItemCat'].str.contains('MAT')) & (df[facility_column].str.contains(gta_pattern)))]
    df_with_middle_gate = pd.concat([df_without_gta, middle_gate], ignore_index=True)
    return df_with_middle_gate
  
# Add "Middle Gate" branch to part_sales
part_sales_with_middle_gate = add_middle_gate_branch(combined_part_sales)

# Filter to include only rows where ItemType is 302 or falls within the 400's range
part_sales_filtered = part_sales_with_middle_gate[(part_sales_with_middle_gate['ItemType'] == 302) | 
                                                  ((part_sales_with_middle_gate['ItemType'] >= 400) & 
                                                   (part_sales_with_middle_gate['ItemType'] < 500))]

# Grouping by 'Facility', 'FacilityName', and 'ItemCat', then counting the occurrences
part_sales_grouped = part_sales_filtered.groupby(['Facility', 'FacilityName', 'ItemCat']).size().reset_index(name='Count')

In [30]:
part_sales_grouped

,Facility,FacilityName,ItemCat,Count
0,AA0,Mount Pearl - 1A Panther Place,CON,1094
1,AA0,Mount Pearl - 1A Panther Place,CRA,21
2,AA0,Mount Pearl - 1A Panther Place,FTY,6
3,AA0,Mount Pearl - 1A Panther Place,MAR,4
4,AA0,Mount Pearl - 1A Panther Place,MAT,452
...,...,...,...,...
382,VJ0,Langley - E. 198th Street,ONH,1852
383,VJ0,Langley - E. 198th Street,POW,7
384,VL0,Tundra Fort St. John - Airport,MNA,217
385,VL0,Tundra Fort St. John - Airport,PWP,7


In [31]:
# Append suffixes to duplicate columns in M3 distribution data
distribution_data.columns = pd.Series(distribution_data.columns).apply(lambda x: x + '_dup' if distribution_data.columns.duplicated().any() else x)

# Append suffixes to duplicate columns in AX distribution data
distribution_data_AX.columns = pd.Series(distribution_data_AX.columns).apply(lambda x: x + '_dup' if distribution_data_AX.columns.duplicated().any() else x)

# Display column names to verify uniqueness
print("Columns in M3 distribution (unique):")
print(distribution_data.columns.tolist())

print("Columns in AX distribution (unique):")
print(distribution_data_AX.columns.tolist())


Columns in M3 distribution (unique):
['Textbox100', 'Textbox40', 'company', 'CompanyName', 'division1', 'DivisionName1', 'Deliverynumber1', 'Direction', 'DeliveryStatus', 'StockTransactionType1', 'FromWhse', 'FromWarehouseName', 'ToWhse', 'ToWarehouseName', 'facility', 'FacilityName', 'DestFacility', 'DestFacilityName', 'FromLocation', 'ToLocation', 'referenceordercategory', 'OrderNumber', 'OrderLine', 'Orderlinelinesuffix', 'ordertype', 'ordertypeDesc', 'MRRORN', 'Transaction_Date', 'deliverymethod', 'addressnumberMAADID', 'companynameMACONM', 'addressline1MAADR1', 'addressline2MAADR2', 'addressline3MAADR3', 'addressline4MAADR4', 'PostalcodeMAPONO', 'countryMACSCD', 'ItemNumber', 'lotnumber1', 'RegNbr', 'itemdesc1', 'itemdesc2', 'ItemType', 'ItemTypeDesc', 'Mfr', 'MfrDesc', 'ItemCategory', 'ItemCategoryDesc', 'SerialCategory', 'SerialCategoryDesc', 'TransactionQuantityBasic', 'InventoryAccountingPrice', 'ExtCost']
Columns in AX distribution (unique):
['Textbox28', 'Textbox24', 'Textbo

### Python Code for Merging the Dist. Sales with a Left Join

In [32]:
# Rename AX columns to match M3 columns
distribution_data_ax_renamed = distribution_data_AX.rename(columns={
    'Textbox24': 'Textbox40',
    'Textbox30': 'company',
    'Textbox9': 'CompanyName',
    'Textbox13': 'division1',
    'Textbox15': 'DivisionName1',
    'Textbox19': 'Deliverynumber1',
    'textbox65': 'Direction',
    'Textbox1': 'DeliveryStatus',
    'Textbox3': 'StockTransactionType1',
    'Textbox26': 'FromWarehouseName',
    'TransDate': 'Transaction_Date',
    'Voucher1': 'facility',
    'FromWhseDesc': 'FacilityName',
    'ToWhseDdesc': 'ToWarehouseName',
    'textbox62': 'ToLocation',
    'Vendor': 'Vendor',
    'VendorName': 'VendorName',
    'ShippedQuantity': 'TransactionQuantityBasic',
    'QTYRECEIVED': 'ExtCost'
})

# Select relevant columns from AX to match the M3 dataset
common_columns_distribution = [
     'FromWarehouseName',  'ToWarehouseName', 'FacilityName', 'ToLocation', 'Transaction_Date', 'TransactionQuantityBasic', 'ExtCost'
]

distribution_data_ax_renamed = distribution_data_ax_renamed[common_columns_distribution]

In [33]:
# Merge M3 and AX distribution data using a left join on relevant columns
combined_distribution_data = pd.merge(
    distribution_data, 
    distribution_data_ax_renamed, 
    on=['FromWarehouseName',  'ToWarehouseName', 'FacilityName', 'ToLocation', 'Transaction_Date'],
    how='left',
    suffixes=('_m3', '_ax')
)

# Display combined columns to check if they match
print("Combined Columns:")
print(combined_distribution_data.columns.tolist())

Combined Columns:
['Textbox100', 'Textbox40', 'company', 'CompanyName', 'division1', 'DivisionName1', 'Deliverynumber1', 'Direction', 'DeliveryStatus', 'StockTransactionType1', 'FromWhse', 'FromWarehouseName', 'ToWhse', 'ToWarehouseName', 'facility', 'FacilityName', 'DestFacility', 'DestFacilityName', 'FromLocation', 'ToLocation', 'referenceordercategory', 'OrderNumber', 'OrderLine', 'Orderlinelinesuffix', 'ordertype', 'ordertypeDesc', 'MRRORN', 'Transaction_Date', 'deliverymethod', 'addressnumberMAADID', 'companynameMACONM', 'addressline1MAADR1', 'addressline2MAADR2', 'addressline3MAADR3', 'addressline4MAADR4', 'PostalcodeMAPONO', 'countryMACSCD', 'ItemNumber', 'lotnumber1', 'RegNbr', 'itemdesc1', 'itemdesc2', 'ItemType', 'ItemTypeDesc', 'Mfr', 'MfrDesc', 'ItemCategory', 'ItemCategoryDesc', 'SerialCategory', 'SerialCategoryDesc', 'TransactionQuantityBasic_m3', 'InventoryAccountingPrice', 'ExtCost_m3', 'TransactionQuantityBasic_ax', 'ExtCost_ax']


### Add the "Middle Gate" Branch After Merge - DIST data

In [34]:
# Define the list of cities in the GTA region
gta_facilities = ["Toronto", "Mississauga", "Brampton", "Vaughan", "Markham", "Richmond Hill", "Oakville", "Burlington", "Pickering", "Ajax", "Whitby"]

# Function to add the "Middle Gate" branch based on FacilityName
def add_middle_gate_branch(df, facility_column='FacilityName'):
    gta_pattern = '|'.join(gta_facilities)
    middle_gate = df[(df[facility_column].str.contains(gta_pattern))]
    middle_gate.loc[:, facility_column] = 'Middle Gate'  # Use .loc to avoid SettingWithCopyWarning
    df_with_middle_gate = pd.concat([df, middle_gate], ignore_index=True)
    return df_with_middle_gate

# Add "Middle Gate" branch to combined distribution data
combined_distribution_data_with_middle_gate = add_middle_gate_branch(combined_distribution_data, facility_column='FacilityName')

# Remove original rows that make up the "Middle Gate" branch
combined_distribution_data_final = combined_distribution_data_with_middle_gate[~((combined_distribution_data_with_middle_gate['FacilityName'].str.contains('|'.join(gta_facilities))))]


In [35]:
combined_distribution_data_final

,Textbox100,Textbox40,company,CompanyName,division1,DivisionName1,Deliverynumber1,Direction,DeliveryStatus,StockTransactionType1,...,MfrDesc,ItemCategory,ItemCategoryDesc,SerialCategory,SerialCategoryDesc,TransactionQuantityBasic_m3,InventoryAccountingPrice,ExtCost_m3,TransactionQuantityBasic_ax,ExtCost_ax
0,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1301703,Delivered,90,51,...,HYSTER,MAT,MATERIAL HANDLING (EQUIP),NaN,NaN,1,$45.44,$45.44,NaN,NaN
1,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1449993,Delivered,90,51,...,HYSTER,MAT,MATERIAL HANDLING (EQUIP),NaN,NaN,1,$7.90,$7.90,NaN,NaN
2,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1466774,Delivered,90,51,...,HYSTER,MAT,MATERIAL HANDLING (EQUIP),NaN,NaN,4,$12.42,$49.66,NaN,NaN
3,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1422172,Delivered,90,51,...,HYSTER,MAT,MATERIAL HANDLING (EQUIP),NaN,NaN,2,$41.28,$82.56,NaN,NaN
4,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1469279,Delivered,90,51,...,HYSTER,MAT,MATERIAL HANDLING (EQUIP),NaN,NaN,2,$42.37,$84.74,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57199,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1506082,Received,90,51,...,HITACHI LOADER,CON,CONSTRUCTION,CON,CONSTRUCTION,1,"$92,282.71","$92,282.71",NaN,NaN
57200,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1522716,Received,90,51,...,HITACHI LOADER,CON,CONSTRUCTION,CON,CONSTRUCTION,1,"$7,155.45","$7,155.45",NaN,NaN
57201,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1381019,Received,90,51,...,HITACHI LOADER,CON,CONSTRUCTION,NaN,NaN,2,$0.30,$0.60,NaN,NaN
57202,Unit Cost CAD,Total Cost CAD,100,100 PROD (PRD),WLT,WAJAX (WLT),1459394,Received,90,51,...,HITACHI LOADER,CON,CONSTRUCTION,CON,CONSTRUCTION,1,"$8,376.00","$8,376.00",NaN,NaN


### Python Code for Merging the Purchase Orders with a Left Join

In [39]:
print("purchase orders M3 Columns:")
print(purchase_orders.columns.tolist())

print("purchase orders AX Columns:")
print(purchase_orders_AX.columns.tolist())

purchase orders M3 Columns:
['RunDate', 'FromDate', 'ToDate', 'Company', 'Div', 'Facility', 'FacilityDesc', 'Whse', 'WarehouseDesc', 'Supplier', 'SupplierName', 'PurchaseOrder', 'POLowStat', 'POHighStat', 'POOrderDate', 'POEntryDate', 'PODueDate', 'POReqDlvDate', 'YourReference', 'POFirstPrintoutDate', 'OrderType', 'OrderTypeDesc', 'POLine', 'POLineSub', 'POLineLowStat', 'POLineHighStat', 'Buyer', 'BuyerName', 'GoodsReceivingMethod', 'GoodsReceivingMethodDesc', 'POCat', 'POLineEntryDate', 'POLineReqDlvDate', 'POLineConfirmedDlvDate', 'POLinePrevConfirmedDlvDate', 'POLineReceiptDate', 'ItemNumber', 'ItemNameOnPOLine', 'ItemDescOnPOLine', 'SerialLotNbr', 'EquipCondition', 'ItemDesc', 'ItemDesc2', 'ItemType', 'ItemTypeDesc', 'ItemGrp', 'ItemGrpDesc', 'ItemCat', 'ItemCatDesc', 'EquipCat', 'EquipCatDesc', 'POUOM', 'POQty', 'POCur', 'PuchasePrice', 'TotDiscount', 'NetPurchasePrice', 'ExtendedPrice', 'RecvQty', 'InvoicedQty', 'RejectedQty', 'LeadTime', 'ActualLeadTime', 'RefOrdCat', 'RefOrdNb

In [40]:
# Rename AX columns to match M3 columns
purchase_orders_ax_renamed = purchase_orders_AX.rename(columns={
    'Vendor Account': 'Supplier',
    'Vendor Name': 'SupplierName',
    'Warehouse Code': 'Whse',
    'Warehouse Name': 'WarehouseDesc',
    'Purch Id': 'PurchaseOrder',
    'PO Line Number': 'POLine',
    'PO Line Created Date': 'POOrderDate',
    'PO Line Confirm Dlv Date': 'PODueDate',
    'Item Id - Item': 'ItemNumber',
    'Item Desc - Item': 'ItemDescOnPOLine',
    'Received Qty': 'RecvQty',
    'PO Line Value$': 'ExtendedPrice'
})

# Select relevant columns from AX to match the M3 dataset
common_columns_po = ['Supplier', 'SupplierName', 'Whse', 'WarehouseDesc', 'PurchaseOrder', 'POLine', 'POOrderDate', 'PODueDate', 'ItemNumber', 'ItemDescOnPOLine', 'RecvQty', 'ExtendedPrice']
purchase_orders_ax_renamed = purchase_orders_ax_renamed[common_columns_po]

# Merge M3 and AX purchase orders data using a left join on 'PurchaseOrder' and 'Supplier'
combined_purchase_orders = pd.merge(purchase_orders, purchase_orders_ax_renamed, on=['Supplier', 'SupplierName', 'Whse', 'WarehouseDesc', 'PurchaseOrder', 'POLine', 'ItemNumber', 'ItemDescOnPOLine'], how='left', suffixes=('_m3', '_ax'))

# Display combined columns to check if they match
print("Combined Columns:")
print(combined_purchase_orders.columns.tolist())

C:\Users\tmanett\AppData\Local\Temp\ipykernel_15884\834632166.py:22: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  combined_purchase_orders = pd.merge(purchase_orders, purchase_orders_ax_renamed, on=['Supplier', 'SupplierName', 'Whse', 'WarehouseDesc', 'PurchaseOrder', 'POLine', 'ItemNumber', 'ItemDescOnPOLine'], how='left', suffixes=('_m3', '_ax'))


Combined Columns:
['RunDate', 'FromDate', 'ToDate', 'Company', 'Div', 'Facility', 'FacilityDesc', 'Whse', 'WarehouseDesc', 'Supplier', 'SupplierName', 'PurchaseOrder', 'POLowStat', 'POHighStat', 'POOrderDate_m3', 'POEntryDate', 'PODueDate_m3', 'POReqDlvDate', 'YourReference', 'POFirstPrintoutDate', 'OrderType', 'OrderTypeDesc', 'POLine', 'POLineSub', 'POLineLowStat', 'POLineHighStat', 'Buyer', 'BuyerName', 'GoodsReceivingMethod', 'GoodsReceivingMethodDesc', 'POCat', 'POLineEntryDate', 'POLineReqDlvDate', 'POLineConfirmedDlvDate', 'POLinePrevConfirmedDlvDate', 'POLineReceiptDate', 'ItemNumber', 'ItemNameOnPOLine', 'ItemDescOnPOLine', 'SerialLotNbr', 'EquipCondition', 'ItemDesc', 'ItemDesc2', 'ItemType', 'ItemTypeDesc', 'ItemGrp', 'ItemGrpDesc', 'ItemCat', 'ItemCatDesc', 'EquipCat', 'EquipCatDesc', 'POUOM', 'POQty', 'POCur', 'PuchasePrice', 'TotDiscount', 'NetPurchasePrice', 'ExtendedPrice_m3', 'RecvQty_m3', 'InvoicedQty', 'RejectedQty', 'LeadTime', 'ActualLeadTime', 'RefOrdCat', 'RefOrd

In [ ]:
# Function to add the "Middle Gate" branch based on FacilityName
def add_middle_gate_branch(df, facility_column='FacilityDesc'):
    gta_pattern = '|'.join(gta_facilities)
    middle_gate = df[(df['ItemCat'].str.contains('MAT')) & (df[facility_column].str.contains(gta_pattern))]
    middle_gate.loc[:, facility_column] = 'Middle Gate'  # Use .loc to avoid SettingWithCopyWarning
    df_with_middle_gate = pd.concat([df, middle_gate], ignore_index=True)
    return df_with_middle_gate

# Add "Middle Gate" branch to the combined data
combined_purchase_orders_with_middle_gate = add_middle_gate_branch(combined_purchase_orders)

# Filter to include only rows where ItemType is 302 or falls within the 400's range
combined_purchase_orders_filtered = combined_purchase_orders_with_middle_gate[(combined_purchase_orders_with_middle_gate['ItemType'] == 302) | 
                                                                              ((combined_purchase_orders_with_middle_gate['ItemType'] >= 400) & 
                                                                               (combined_purchase_orders_with_middle_gate['ItemType'] < 500))]

# Grouping by 'Facility', 'SupplierName', and 'ItemCat', then counting the occurrences
purchase_orders_grouped = combined_purchase_orders_filtered.groupby(['Facility', 'FacilityDesc', 'SupplierName', 'ItemCat']).size().reset_index(name='Count')


KeyError: 'Facility'